### Text-to-SQL Bot
#### "Talk to Your Database"  

Abhi tumne unstructured data (text/articles) ke sath chat ki. Industry mein asli paisa Structured Data (SQL Databases) mein hai. Companies chahti hain ke unka manager seedha puche: "Pichle mahene kitni sales hui?" aur bot database se query kar ke jawab de.

- Concept: 
    User English mein sawal puchega -> Bot usay SQL Query mein convert karega -> Database par run karega -> Result ko wapis English mein batayega.

##### Kya seekho ge:

LangChain SQL Toolkit: Database se kaise connect karte hain.

Prompt Engineering for Code: LLM se accurate SQL kaise likhwana hai.

Security: Injection attacks se kaise bachen.



In [58]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_community.utilities import SQLDatabase
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

load_dotenv()

api_key = os.getenv("HUGGINGFACE_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")



In [30]:
question = input("What is your question? ")

In [ ]:
db_user = "root"          
db_password = ""          
db_host = "localhost"     
db_name = "ok"    

mysql_uri = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:3306/{db_name}"

try:
    db = SQLDatabase.from_uri(mysql_uri)
    print("✅ Database Connected Successfully!")
    print("Dialect:", db.dialect)
    print("Tables found:", db.get_usable_table_names())
except Exception as e:
    print("❌ Connection Error! Check your details.")
    print(e)

✅ Database Connected Successfully!
Dialect: mysql
Tables found: ['cache', 'cache_locks', 'categories', 'company_settings', 'customer_advances', 'customer_credit_payments', 'customers', 'expense_categories', 'expenses', 'failed_jobs', 'job_batches', 'jobs', 'migrations', 'panaflex_specs', 'password_reset_tokens', 'payments', 'pending_payments', 'permissions', 'products', 'purchase_items', 'purchases', 'register_sessions', 'role_permissions', 'roles', 'sale_items', 'sale_return_items', 'sale_returns', 'sales', 'sessions', 'stock_adjustments', 'stock_batches', 'stock_moves', 'suppliers', 'units', 'user_permissions', 'users']


In [59]:
llm = HuggingFaceEndpoint(
    # repo_id="deepseek-ai/DeepSeek-R1",
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    huggingfacehub_api_token=api_key,
)

model = ChatHuggingFace(llm=llm, temperature=0.9)


model_02 = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, api_key=google_api_key)

In [60]:
toolkit = SQLDatabaseToolkit(db=db, llm=model_02)

agent = create_sql_agent(
    llm=model_02,
    toolkit=toolkit,
    verbose=True,  
    agent_type="zero-shot-react-description",  
    handle_parsing_errors=True,  
    max_iterations=10,  
    early_stopping_method="generate",
    agent_executor_kwargs={
        "return_intermediate_steps": True 
    },
    prefix="""You are a senior SQL expert and database analyst.

Your task is to generate a syntactically correct, optimized SQL query
based ONLY on the provided database schema.

STRICT RULES:

1. Use ONLY tables and columns mentioned in the schema.
2. Do NOT assume columns that are not listed.
3. If question is ambiguous, choose the most logical interpretation.
4. Always prefer aggregation functions when user asks for totals, averages, counts.
5. Use proper JOIN conditions if multiple tables are required.
6. Do NOT generate explanations.
7. Use LIMIT 10 if user asks for listing data without specifying limit.
8. Always handle date-based filtering carefully.
9. If query cannot be answered using schema, return:
   -- CANNOT_GENERATE_SQL"""
)

In [ ]:
response = agent.invoke({"input": question})
print(response["output"])